# Convolutional Neural Network dengan PyTorch

### Dataset

Menggunakan library dari google colab, kami bisa menghubungkan langsung dari shared google drive ke google colab ini. Proses ini sangat menghemat waktu karna tidak perlu upload berulang setiap memulai runtime.

Sumber dataset ada tiga yaitu instagram, google image, dan foto sendiri. 
- Untuk yang dari instagram memiliki format penamaan (username ig)-(tanggal didownload)-(file keberapa)
- Untuk yang foto sendiri penamaan mengikuti format dari HP, umumnya penamaan file diawali photos... atau FILE...
- Sisanya adalah penamaan yang tak beraturan bersumber dari google images.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=False)

Mounted at /content/gdrive/


### Packages

In [ ]:
import torch as pt # library utama pytorch
import numpy as np # numerical python
import pandas as pd # bermain dataset
import copy # untuk copy2 an

# libray lain pytorch untuk membangun arsitektur
import torch.nn as nn 
import torch.nn.functional as F

# library dar pytorch untuk pengelolaan dataset khususnya data citra
from torchvision import transforms, datasets, models

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

### Preprocessing
Pada bagian ini kita akan memperbaiki data agar enak dibaca CNN

Input CNN sebesar 244x244, maka agar tidak kehilangan informasi, foto di resize menjadi 256 lalu di centre crop sebesar 224x224.

In [ ]:
train_dir = "/content/gdrive/Shareddrives/Tubes DL/Train"
test_dir = "/content/gdrive/Shareddrives/Tubes DL/Test"

# fungsi transform bertugas untuk "mengedit" data yang masuk
# macam-macam perintah yang bisa dilakukan bisa cek di sini: https://pytorch.org/docs/stable/torchvision/transforms.html

train_transform = transforms.Compose([ # proses transform dilakukan secara berurutan
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(), # 2. di ubah ke tensor
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize(256), 
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# load dataset dengan format folder yang sesuai lalu aplikasikan transformasi ke data yang dibaca
# hasil dari proses ini adalah sebuah list berelemen tuple (tensor_citra, label) disebut dengan objek datasets
train_img = datasets.ImageFolder(train_dir, transform=train_transform)
test_img = datasets.ImageFolder(test_dir, transform=test_transform)

In [ ]:
trainloaders = pt.utils.data.DataLoader(train_img, batch_size=16, shuffle=True)
testloaders = pt.utils.data.DataLoader(test_img, batch_size=1, shuffle=True)

In [ ]:
# Informasi dari dataset kita
print("train_img type   :",type(train_img)) # objek datasets

print("train_img length :",len(train_img)) # 515 data train
print("test_img length :",len(test_img)) # 66 data test

print("train_img classes:",train_img.classes) # nama kelas
print("train_img classes:",train_img.class_to_idx) # nama kelas

print("train_img[0] type:",type(train_img[0])) # tuple (A, B)
print("train_img[0][0]  :",train_img[0][0].size()) # tensor citra
print("train_img[0][1]  :",train_img[0][1]) # label

# variabel banyak data
len_train = len(train_img)
len_test = len(test_img)

train_img type   : torchvision.datasets.folder.ImageFolder
train_img length : 477
test_img length : 90
train_img classes: ['KAIN', 'MEDIS', 'SCUBA']
train_img classes: {'KAIN': 0, 'MEDIS': 1, 'SCUBA': 2}
train_img[0] type: <class 'tuple'>
train_img[0][0]  : torch.Size([3, 224, 224])
train_img[0][1]  : 0


### Buat Arsitektur
Arsitektur menggunakan pre-trained model residual networks dengan model structure resnet50 karena errornya tidak jauh berbeda dengan resnet152 namun size file pth yang dihasilkan relatif kecil.

In [ ]:
modelkita = models.resnet50(pretrained=True)

#freezing
for param in modelkita.parameters():
    param.requires_grad = False

# hapus layer terakhir
modelkita.fc = nn.Linear(2048, 3) #2048 dilihat dari jumlah layer kedua sebelum terakhir

modelkita = modelkita.cuda()
print(modelkita)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
optimizer = pt.optim.Adagrad(modelkita.parameters(), lr=0.01) # algoritma training
criterion = nn.CrossEntropyLoss() # rumus error

### Mulai Training

Agar tidak overfitting, kami memilih menggunakan 25 epoch, berhenti sebelum akurasi train mencapai 90% 

In [ ]:
modelkita.train()
for i in range(25): # 25 epoch

    rata2error = 0 # untuk menghitung error ("seberapa salah")
    jumlahbenar = 0 # untuk menghitung akurasi ("seberapa benar")

    for image, label in trainloaders: # iterasi semua data
        
        image = image.cuda()
        label = label.cuda()

        # forward
        Y = modelkita( image )
        E = criterion( Y, label )
        rata2error += E

        prediksi = pt.max(Y, dim=1).indices
        for i in range(len(prediksi)):
            if prediksi[i]==label[i]:
                jumlahbenar+=1
        
        # backward
        optimizer.zero_grad()
        E.backward()       

        # update
        optimizer.step() 

    print(rata2error/len_train, jumlahbenar/len_train)

tensor(0.0977, device='cuda:0', grad_fn=<DivBackward0>) 0.5471698113207547
tensor(0.0469, device='cuda:0', grad_fn=<DivBackward0>) 0.6876310272536688
tensor(0.0405, device='cuda:0', grad_fn=<DivBackward0>) 0.7526205450733753
tensor(0.0339, device='cuda:0', grad_fn=<DivBackward0>) 0.7924528301886793
tensor(0.0326, device='cuda:0', grad_fn=<DivBackward0>) 0.7945492662473794
tensor(0.0321, device='cuda:0', grad_fn=<DivBackward0>) 0.80083857442348
tensor(0.0306, device='cuda:0', grad_fn=<DivBackward0>) 0.8155136268343816
tensor(0.0282, device='cuda:0', grad_fn=<DivBackward0>) 0.8301886792452831
tensor(0.0269, device='cuda:0', grad_fn=<DivBackward0>) 0.8427672955974843
tensor(0.0264, device='cuda:0', grad_fn=<DivBackward0>) 0.8469601677148847
tensor(0.0261, device='cuda:0', grad_fn=<DivBackward0>) 0.8553459119496856
tensor(0.0255, device='cuda:0', grad_fn=<DivBackward0>) 0.8616352201257862
tensor(0.0252, device='cuda:0', grad_fn=<DivBackward0>) 0.8511530398322851
tensor(0.0250, device='cuda

### Testing


In [ ]:
modelkita.eval()

with pt.no_grad():
    rata2error = 0 # untuk menghitung error ("seberapa salah")
    jumlahbenar = 0 # untuk menghitung akurasi ("seberapa benar")

    for image, label in testloaders: # iterasi semua data
        
        image = image.cuda()
        label = label.cuda()

        # forward
        Y = modelkita( image )
        E = criterion( Y, label )
        rata2error += E

        prediksi = pt.max(Y, dim=1).indices
        for i in range(len(prediksi)):
            if prediksi[i]==label[i]:
                jumlahbenar+=1

    print(rata2error/len_test, jumlahbenar/len_test)

tensor(0.3460, device='cuda:0') 0.9


### Saving Model

In [ ]:
pt.save(modelkita, "modelkita_88_90.pth")

#### Copy model ke Shared Drive

File pth terbilang cukup besar, butuh banyak waktu untuk download dan upload ke shared drive agar bisa digunakan bersama teman kelompok yang lain, maka dari itu kami menggunakan command cp (command linux) untuk meng-copy files dari collab langsung ke google drive


In [ ]:
!cp "/content/modelkita_88_90.pth" "/content/gdrive/Shareddrives/Tubes DL"